In [2]:
import spacy
from date_spacy import find_dates
import components

c:\Users\dd758\Formation_IA_Greta\Vocal_Weather\Vocal-Weather-2025\tests\components.py:90: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
c:\Users\dd758\Formation_IA_Greta\Vocal_Weather\Vocal-Weather-2025\tests\components.py:100: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
c:\Users\dd758\Formation_IA_Greta\Vocal_Weather\Vocal-Weather-2025\tests\components.py:115: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """


In [3]:
nlp = spacy.blank('fr')

# Add the component to the pipeline
nlp.add_pipe('find_dates')

<function components.find_dates(doc)>

In [4]:
doc = nlp("""The event is scheduled for 25th August 2023.
          We also have a meeting on 10 September and another one on the twelfth of October and a
          final one on January fourth.""")
for ent in doc.ents:
    if ent.label_ == "DATE":
        print(f"Text: {ent.text} -> Parsed Date: {ent._.date}")

In [5]:


doc = nlp("""L'événement est prévu pour le 25 août 2023.
          Nous avons également une réunion le 10 septembre et une autre le douze octobre et une
          dernière le quatre janvier""")
for ent in doc.ents:
    if ent.label_ == "DATE":
        print(f"Text: {ent.text} -> Parsed Date: {ent._.date}")


Text: 25 août 2023 -> Parsed Date: 2023-08-25 00:00:00
Text: 10 septembre -> Parsed Date: 2025-09-10 00:00:00
Text: douze octobre -> Parsed Date: 2025-10-12 00:00:00
Text: quatre janvier -> Parsed Date: 2025-01-04 00:00:00


## Test numéro 2

In [6]:
import re
from spacy.tokens import Span
from spacy.language import Language
from spacy.util import filter_spans
import dateparser

ordinal_to_number = {
    "premier": "1", "deux": "2", "trois": "3", "quatre": "4", "cinq": "5",
    "six": "6", "sept": "7", "huit": "8", "neuf": "9", "dix": "10",
    "onze": "11", "douze": "12", "treize": "13", "quatorze": "14",
    "quinze": "15", "seize": "16", "dix-sept": "17", "dix-huit": "18",
    "dix-neuvième": "19", "vingt": "20", "vingt-et-un": "21", "vingt-deux": "22",
    "vingt-trois": "23", "vingt-quatre": "24", "vingt-cinq": "25", "vingt-six": "26",
    "vingt-sept": "27", "vingt-huit": "28", "vingt-neuf": "29", "trente": "30", 
    "trente-et-un": "31"
}

@Language.component("find_dates")
def find_dates(doc):
    # Définir une extension de date sur le span
    Span.set_extension("date", default=None, force=True)

    # Ordinaux
    ordinals = [
        "premier", "deux", "trois", "quatre", "cinq",
        "six", "sept", "huit", "neuf", "dix",
        "onze", "douze", "treize", "quatorze",
        "quinze", "seize", "dix-sept", "dix-huit",
        "dix-neuf", "vingt", "vingt-et-un", "vingt-deux",
        "vingt-trois", "vingt-quatre", "vingt-cinq", "vingt-six",
        "vingt-sept", "vingt-huit", "vingt-neuf", "trente", "trente-et-un"  
    ]
    
    ordinal_pattern = r"\b(?:" + "|".join(ordinals) + r")\b"

    # Un modèle regex pour capturer une variété de formats de date
    date_pattern = r"""
        # Jour-Mois-Année
        (?:
            \d{1,2}(?:er|ème)?          # Jour avec suffixe optionnel er, ème
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Jour/Mois/Année
        (?:
            \d{1,2}                     # Jour
            [/-]
            \d{1,2}                     # Mois
            (?:                         # Année est optionnelle
                [/-]
                \d{2,4}                 # Année
            )?
        )
        |
        # Année-Mois-Jour
        (?:
            \d{4}                       # Année
            [-/]
            \d{1,2}                     # Mois
            [-/]
            \d{1,2}                     # Jour
        )
        |
        # Mois-Jour-Année
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            \s+
            \d{1,2}(?:er|ème)?          # Jour avec suffixe optionnel er, ème
            (?:                         # Année est optionnelle
                ,?
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Mois-Année
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            \s+
            \d{4}                       # Année
        )
        |
        # Ordinal-Jour-Mois-Année
        (?:
            """ + ordinal_pattern + """
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        (?:
            """ + ordinal_pattern + """
            \s+
            de
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]*  # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Mois Ordinal
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]*  # Nom du mois
            \s+
            """ + ordinal_pattern + """
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
    """
    matches = list(re.finditer(date_pattern, doc.text, re.VERBOSE))
    new_ents = []
    for match in matches:
        start_char, end_char = match.span()
        # Convertir les offsets de caractères en offsets de tokens
        start_token = None
        end_token = None
        for token in doc:
            if token.idx == start_char:
                start_token = token.i
            if token.idx + len(token.text) == end_char:
                end_token = token.i
        if start_token is not None and end_token is not None:
            hit_text = doc.text[start_char:end_char]
            parsed_date = dateparser.parse(hit_text, languages=["fr"])
            if parsed_date:  # S'assurer que la chaîne correspondante est une date valide
                ent = Span(doc, start_token, end_token + 1, label="DATE")
                ent._.date = parsed_date
                new_ents.append(ent)
            else:
                # Remplacer chaque ordinal dans hit_text par sa représentation numérique
                for ordinal, number in ordinal_to_number.items():
                    hit_text = hit_text.replace(ordinal, number)

                # Supprimer le mot "de" de hit_text
                new_date = hit_text.replace(" de ", " ")

                parsed_date = dateparser.parse(new_date, languages=["fr"])
                ent = Span(doc, start_token, end_token + 1, label="DATE")
                ent._.date = parsed_date
                new_ents.append(ent)
    # Combiner les nouvelles entités avec les entités existantes, en s'assurant qu'il n'y a pas de chevauchement
    doc.ents = list(doc.ents) + new_ents
    
    return doc

<>:90: SyntaxWarning: invalid escape sequence '\s'
<>:100: SyntaxWarning: invalid escape sequence '\s'
<>:115: SyntaxWarning: invalid escape sequence '\s'
<>:90: SyntaxWarning: invalid escape sequence '\s'
<>:100: SyntaxWarning: invalid escape sequence '\s'
<>:115: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dd758\AppData\Local\Temp\ipykernel_17772\3485255653.py:90: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
C:\Users\dd758\AppData\Local\Temp\ipykernel_17772\3485255653.py:100: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
C:\Users\dd758\AppData\Local\Temp\ipykernel_17772\3485255653.py:115: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """


In [13]:
import spacy
from spacy.tokens import Span
import dateparser
import re

# Charger le modèle de langue française
nlp = spacy.load("fr_core_news_md")

# Définir une extension de date sur le span
Span.set_extension("date", default=None, force=True)

# Fonction pour trouver les dates
def find_dates(doc):
    ordinal_to_number = {
        "premier": "1", "deux": "2", "trois": "3", "quatre": "4", "cinq": "5",
        "six": "6", "sept": "7", "huit": "8", "neuf": "9", "dix": "10",
        "onze": "11", "douze": "12", "treize": "13", "quatorze": "14",
        "quinze": "15", "seize": "16", "dix-sept": "17", "dix-huit": "18",
        "dix-neuvième": "19", "vingt": "20", "vingt-et-un": "21", "vingt-deux": "22",
        "vingt-trois": "23", "vingt-quatre": "24", "vingt-cinq": "25", "vingt-six": "26",
        "vingt-sept": "27", "vingt-huit": "28", "vingt-neuf": "29", "trente": "30", 
        "trente-et-un": "31"
    }

    ordinal_pattern = r"\b(?:" + "|".join(ordinal_to_number.keys()) + r")\b"

    date_pattern = r"""
        # Jour-Mois-Année
        (?:
            \d{1,2}(?:er|ème)?          # Jour avec suffixe optionnel er, ème
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Jour/Mois/Année
        (?:
            \d{1,2}                     # Jour
            [/-]
            \d{1,2}                     # Mois
            (?:                         # Année est optionnelle
                [/-]
                \d{2,4}                 # Année
            )?
        )
        |
        # Année-Mois-Jour
        (?:
            \d{4}                       # Année
            [-/]
            \d{1,2}                     # Mois
            [-/]
            \d{1,2}                     # Jour
        )
        |
        # Mois-Jour-Année
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            \s+
            \d{1,2}(?:er|ème)?          # Jour avec suffixe optionnel er, ème
            (?:                         # Année est optionnelle
                ,?
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Mois-Année
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            \s+
            \d{4}                       # Année
        )
        |
        # Ordinal-Jour-Mois-Année
        (?:
            """ + ordinal_pattern + """
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]* # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        (?:
            """ + ordinal_pattern + """
            \s+
            de
            \s+
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]*  # Nom du mois
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
        |
        # Mois Ordinal
        (?:
            (?:janv|févr|mars|avr|mai|juin|juil|août|sept|oct|nov|déc)[a-z]*  # Nom du mois
            \s+
            """ + ordinal_pattern + """
            (?:                         # Année est optionnelle
                \s+
                \d{4}                   # Année
            )?
        )
    """
    matches = list(re.finditer(date_pattern, doc.text, re.VERBOSE))
    new_ents = []
    for match in matches:
        start_char, end_char = match.span()
        start_token = None
        end_token = None
        for token in doc:
            if token.idx == start_char:
                start_token = token.i
            if token.idx + len(token.text) == end_char:
                end_token = token.i
        if start_token is not None and end_token is not None:
            hit_text = doc.text[start_char:end_char]
            parsed_date = dateparser.parse(hit_text, languages=["fr"])
            if parsed_date:
                ent = Span(doc, start_token, end_token + 1, label="DATE")
                ent._.date = parsed_date
                new_ents.append(ent)
            else:
                for ordinal, number in ordinal_to_number.items():
                    hit_text = hit_text.replace(ordinal, number)
                new_date = hit_text.replace(" de ", " ")
                parsed_date = dateparser.parse(new_date, languages=["fr"])
                ent = Span(doc, start_token, end_token + 1, label="DATE")
                ent._.date = parsed_date
                new_ents.append(ent)
    doc.ents = list(doc.ents) + new_ents
    return doc

# Ajouter le composant au pipeline
nlp.add_pipe("find_dates", last=True)

# Texte de test
text = """L'événement est prévu pour le 25 août 2023.
          Nous avons également une réunion le 10 septembre et une autre le douze octobre et une
          dernière le quatre janvier et jeudi suivant"""

# Traiter le texte
doc = nlp(text)

# Afficher les entités trouvées
for ent in doc.ents:
    print(ent.text, ent.label_, ent._.date)


<>:80: SyntaxWarning: invalid escape sequence '\s'
<>:90: SyntaxWarning: invalid escape sequence '\s'
<>:105: SyntaxWarning: invalid escape sequence '\s'
<>:80: SyntaxWarning: invalid escape sequence '\s'
<>:90: SyntaxWarning: invalid escape sequence '\s'
<>:105: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dd758\AppData\Local\Temp\ipykernel_1512\1564231513.py:80: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
C:\Users\dd758\AppData\Local\Temp\ipykernel_1512\1564231513.py:90: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """
C:\Users\dd758\AppData\Local\Temp\ipykernel_1512\1564231513.py:105: SyntaxWarning: invalid escape sequence '\s'
  """ + ordinal_pattern + """


25 août 2023 DATE 2023-08-25 00:00:00
10 septembre DATE 2025-09-10 00:00:00
douze octobre DATE 2025-10-12 00:00:00
quatre janvier DATE 2025-01-04 00:00:00


## Test Api Meteo ; Open Meteo

In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/meteofrance"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m",
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

Coordinates 52.52000045776367°N 13.40999984741211°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m
0,2025-02-20 00:00:00+00:00,-3.0805
1,2025-02-20 01:00:00+00:00,-3.6805
2,2025-02-20 02:00:00+00:00,-3.6305
3,2025-02-20 03:00:00+00:00,-3.7805
4,2025-02-20 04:00:00+00:00,-3.9805
...,...,...
91,2025-02-23 19:00:00+00:00,1.7935
92,2025-02-23 20:00:00+00:00,1.5435
93,2025-02-23 21:00:00+00:00,1.3935
94,2025-02-23 22:00:00+00:00,1.4435


### test numero je sais plus combien 

In [31]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/meteofrance"
params = {
	"latitude": 47.3948,
	"longitude": 0.704,
	"daily": ["temperature_2m_max", "temperature_2m_min", "precipitation_sum", "rain_sum", "wind_speed_10m_max", "wind_speed_10m_min"],
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(4).ValuesAsNumpy()
daily_wind_speed_10m_min = daily.Variables(5).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_speed_10m_min"] = daily_wind_speed_10m_min

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe


Coordinates 47.38999938964844°N 0.6999998092651367°E
Elevation 51.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,wind_speed_10m_max,wind_speed_10m_min
0,2025-02-19 00:00:00+00:00,14.2500,2.8500,0.000000,0.000000,12.481153,5.091168
1,2025-02-20 00:00:00+00:00,15.0500,7.4500,0.200000,0.300000,12.303366,1.297998
2,2025-02-21 00:00:00+00:00,14.9000,8.0000,0.000000,0.000000,23.039999,6.849467
3,2025-02-22 00:00:00+00:00,13.3755,7.5755,6.199999,6.199999,29.957972,4.349896


In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "rain", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "wind_speed_10m", "is_day"],
	"past_minutely_15": 96,
	"forecast_minutely_15": 96,
	"temporal_resolution": "hourly_6",
	"models": "meteofrance_seamless"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
hourly_is_day = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 52.52000045776367°N 13.40999984741211°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                        date  temperature_2m  rain  cloud_cover  \
0  2025-02-24 00:00:00+00:00          3.8195   0.0          0.0   
1  2025-02-24 06:00:00+00:00          2.2195   0.0        100.0   
2  2025-02-24 12:00:00+00:00         12.2195   0.0         99.0   
3  2025-02-24 18:00:00+00:00         11.2195   0.0        100.0   
4  2025-02-25 00:00:00+00:00          8.2195   0.3        100.0   
5  2025-02-25 06:00:00+00:00          7.7195   2.0        100.0   
6  2025-02-25 12:00:00+00:00         10.4695   0.0        100.0   
7  2025-02-25 18:00:00+00:00          8.9695   0.0        100.0   
8  2025-02-26 00:00:00+00:00          7.7195   2.3        100.0   
9  2025-02-26 06:00:00+00:00          7.3695   1.9        100.0   
10 2025-02-26 12:00:00+00:00          7.6435   0.8         82.0   
11 2025-02-26 18:00:00+00:00          3.6435   0.6         42.0   
12 2

In [1]:
import requests
from typing import Tuple

def get_coordinates(city_name: str) -> Tuple[float, float]:
    geocode_url = "https://nominatim.openstreetmap.org/search"
    params = {"q": city_name, "format": "json"}
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(geocode_url, params=params, headers=headers)
    data = r.json()
    if not data:
        raise Exception(f"Ville introuvable : {city_name}")
    lat = float(data[0]["lat"])
    lon = float(data[0]["lon"])
    return lat, lon

In [2]:
get_coordinates("Paris")

(48.8534951, 2.3483915)

In [3]:
get_coordinates("Los Angeles")

(34.0536909, -118.242766)